# RAG Médico con BioMistral - Sistema de Análisis Clínico

**Asignatura:** Transformers del Lenguaje en Salud (UPCH)

## Objetivo

Sistema RAG (Retrieval-Augmented Generation) que:
- ✅ Usa **BioMistral-7B** (LLM médico especializado)
- ✅ **PubMedBERT** para embeddings médicos
- ✅ Se integra con **NER + EntityLinker** existente
- ✅ Búsqueda en knowledge base médica
- ✅ Soporta múltiples backends (HF API, HF Local, Ollama)

## 1️⃣ Instalación de Dependencias

In [1]:
# Instalación de paquetes necesarios
%pip install -q langchain langchain-community langchain-huggingface langchain-text-splitters
%pip install -q faiss-cpu
%pip install -q transformers accelerate
%pip install -q sentence-transformers
%pip install -q python-docx pypdf

print("✅ Dependencias instaladas")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ Dependencias instaladas


## 2️⃣ Importación de Librerías

In [2]:
import json
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Langchain
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
# ✅ CORREGIDO: Usando LangChain Core LCEL
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# Hugging Face
from huggingface_hub import login

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 3️⃣ Configuración de Hugging Face

**IMPORTANTE:** Necesitas un token de Hugging Face (gratuito)
- Ir a: https://huggingface.co/settings/tokens
- Crear token con permisos de lectura
- Pegar abajo

In [4]:
# Configurar tu token de Hugging Face
HF_TOKEN = ""  # ⬅️ TOKEN ELIMINADO POR SEGURIDAD

if HF_TOKEN:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN
    login(token=HF_TOKEN)
    print("✅ Autenticado en Hugging Face")
else:
    print("⚠️ Configura tu token de Hugging Face")

✅ Autenticado en Hugging Face


## 4️⃣ Embeddings Médicos: PubMedBERT

Usamos PubMedBERT porque:
- Entrenado en literatura médica (PubMed)
- Comprende mejor terminología médica
- Mejor retrieval para docs médicos vs embeddings genéricos

In [5]:
print("⏳ Cargando PubMedBERT para embeddings...")

embeddings = HuggingFaceEmbeddings(
    model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Test
test_embedding = embeddings.embed_query("diabetes mellitus treatment")
print(f"✅ PubMedBERT listo: dimensión = {len(test_embedding)}")

⏳ Cargando PubMedBERT para embeddings...


No sentence-transformers model found with name microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract. Creating a new one with mean pooling.


✅ PubMedBERT listo: dimensión = 768


## 5️⃣ Preparar Knowledge Base Médica

Crea una carpeta `medical_knowledge/` y añade documentos médicos:
- Guías clínicas (PDF, TXT)
- Protocolos
- Artículos de referencia

In [6]:
# Crear directorio de ejemplo si no existe
knowledge_dir = Path("medical_knowledge")
knowledge_dir.mkdir(exist_ok=True)

# Documento de ejemplo sobre diabetes
example_doc = knowledge_dir / "diabetes_guideline.txt"
if not example_doc.exists():
    example_doc.write_text("""
GUÍA CLÍNICA: DIABETES MELLITUS TIPO 2

DIAGNÓSTICO:
- Glucosa en ayunas ≥126 mg/dL (7.0 mmol/L)
- HbA1c ≥6.5%
- Glucosa aleatoria ≥200 mg/dL con síntomas

COMPLICACIONES COMUNES:
1. Cardiovasculares (IAM, ICC)
   - DM2 aumenta 2-4x riesgo cardiovascular
   - Asociación frecuente con hipertensión arterial
   
2. Nefropatía diabética
3. Retinopatía
4. Neuropatía

TRATAMIENTO FARMACOLÓGICO:
Primera línea: Metformina 500-850mg
- Contraindicaciones: ICC descompensada, insuficiencia renal
- Efectos adversos: Intolerancia gastrointestinal, acidosis láctica (rara)

HIPERTENSIÓN EN DIABÉTICOS:
Objetivo: PA <140/90 mmHg
Fármacos preferidos:
- IECA (Enalapril, Lisinopril)
- ARA-II (Losartán, Valsartán)
  * Losartán 50-100mg/día
  * Nefroprotector
  * Bien tolerado en diabéticos

SÍNTOMAS DE ALARMA:
- Disnea súbita: considerar ICC (especialmente si DM2 + HTA)
- Dolor torácico: IAM silente común en diabéticos
- Edema en miembros inferiores: ICC o nefropatía

REFERENCIA: Adaptado de ADA Standards of Medical Care in Diabetes 2024
    """)
    print("✅ Documento de ejemplo creado")

# Documento sobre ICC
icc_doc = knowledge_dir / "heart_failure.txt"
if not icc_doc.exists():
    icc_doc.write_text("""
GUÍA: INSUFICIENCIA CARDÍACA CONGESTIVA (ICC)

FACTORES DE RIESGO:
- Hipertensión arterial (causa principal)
- Diabetes Mellitus
- Enfermedad coronaria
- Edad avanzada (>65 años)

PRESENTACIÓN CLÍNICA:
Síntomas:
- Disnea de esfuerzo (síntoma cardinal)
- Ortopnea
- Disnea paroxística nocturna
- Fatiga
- Edema en MMII

Signos:
- Taquicardia
- Crepitantes pulmonares
- Ingurgitación yugular
- Hepatomegalia

DIAGNÓSTICO:
1. Ecocardiograma (Gold standard)
   - Evaluar FE (fracción de eyección)
2. BNP/Pro-BNP
   - BNP >100 pg/mL sugiere ICC
3. Radiografía tórax
   - Cardiomegalia, congestión pulmonar

TRATAMIENTO:
Medidas generales:
- Restricción de sal (<2g/día)
- Control de líquidos
- Ejercicio moderado (cuando esté estable)

Farmacológico:
- Diuréticos (furosemida)
- IECA/ARA-II (continuar si ya los toma)
- ⚠️ METFORMINA: CONTRAINDICADA en ICC descompensada
  * Riesgo de acidosis láctica
  * Suspender temporalmente

PACIENTE DIABÉTICO CON ICC:
- Ajustar hipoglucemiantes (evitar metformina)
- Control estricto de PA
- Monitoreo frecuente

REFERENCIA: ESC Guidelines for Heart Failure 2024
    """)
    print("✅ Documento ICC creado")

print(f"📁 Knowledge base en: {knowledge_dir.absolute()}")

✅ Documento de ejemplo creado
✅ Documento ICC creado
📁 Knowledge base en: /Users/smiljaandreasalazarcortez/PROYECTO_DIPLOMADO/Proyecto_T_L/notebooks/medical_knowledge


## 6️⃣ Cargar y Procesar Documentos

In [7]:
print("📚 Cargando documentos médicos...")

# Cargar documentos
loader = DirectoryLoader(
    str(knowledge_dir),
    glob="**/*.txt",
    loader_cls=TextLoader
)
documents = loader.load()

print(f"✅ Cargados {len(documents)} documentos")

# Split en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Chunks más grandes para contexto médico
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " ", ""]
)
chunks = text_splitter.split_documents(documents)

print(f"✅ Divididos en {len(chunks)} chunks")
print(f"\n📄 Ejemplo de chunk:\n{chunks[0].page_content[:200]}...")

📚 Cargando documentos médicos...
✅ Cargados 2 documentos
✅ Divididos en 6 chunks

📄 Ejemplo de chunk:
GUÍA CLÍNICA: DIABETES MELLITUS TIPO 2

DIAGNÓSTICO:
- Glucosa en ayunas ≥126 mg/dL (7.0 mmol/L)
- HbA1c ≥6.5%
- Glucosa aleatoria ≥200 mg/dL con síntomas

COMPLICACIONES COMUNES:
1. Cardiovasculares ...


## 7️⃣ Crear Vector Database (FAISS)

In [8]:
print("⏳ Creando vector database...")

vector_db = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

print(f"✅ Vector database creada con {len(chunks)} vectores")

# Test de búsqueda
test_query = "tratamiento diabetes hipertensión"
test_results = vector_db.similarity_search(test_query, k=2)

print(f"\n🔍 Test búsqueda: '{test_query}'")
print(f"Resultado 1: {test_results[0].page_content[:150]}...")

⏳ Creando vector database...
✅ Vector database creada con 6 vectores

🔍 Test búsqueda: 'tratamiento diabetes hipertensión'
Resultado 1: SÍNTOMAS DE ALARMA:
- Disnea súbita: considerar ICC (especialmente si DM2 + HTA)
- Dolor torácico: IAM silente común en diabéticos
- Edema en miembros...


## 8️⃣ Configurar BioMistral (LLM Médico)

**BioMistral-7B:** Modelo Mistral fine-tuned en datos médicos
- Estado del arte en benchmarks médicos
- Multilingüe (español médico incluido)

In [10]:
print("⏳ Configurando BioMistral...")

llm = HuggingFaceEndpoint(
    repo_id="BioMistral/BioMistral-7B",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.3,  # Más bajo para respuestas médicas precisas
    top_p=0.9,
    repetition_penalty=1.1,
    huggingfacehub_api_token=HF_TOKEN
)

# Test simple
test_response = llm.invoke("What is diabetes mellitus?")
print(f"✅ BioMistral listo\n\n📝 Test: {test_response[:200]}...")

⏳ Configurando BioMistral...


StopIteration: 

## 9️⃣ Prompt Template Médico

In [ ]:
# Prompt especializado para análisis clínico
medical_prompt_template = """You are a specialized medical assistant. You will analyze clinical information and provide evidence-based insights.

MEDICAL KNOWLEDGE (from guidelines):
{context}

CLINICAL QUESTION:
{question}

INSTRUCTIONS:
1. Base your answer ONLY on the provided medical knowledge
2. Provide differential diagnoses when relevant
3. Include management recommendations from guidelines
4. Highlight any contraindications or alerts
5. Cite specific guideline references

ANSWER:
"""

MEDICAL_PROMPT = PromptTemplate(
    template=medical_prompt_template,
    input_variables=["context", "question"]
)

print("✅ Prompt template configurado")

## 🔟 Pipeline RAG Completo

In [ ]:
print("⏳ Construyendo pipeline RAG...")

# Crear retriever
retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Top 3 documentos más relevantes
)

# Crear chain RAG
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": MEDICAL_PROMPT},
    return_source_documents=True
)

print("✅ Pipeline RAG listo")

## 1️⃣1️⃣ Función Helper para Consultas Médicas

In [ ]:
def medical_query(question, verbose=True):
    """
    Realiza una consulta médica al sistema RAG.
    
    Args:
        question: Pregunta clínica
        verbose: Mostrar documentos recuperados
    
    Returns:
        Respuesta del LLM con referencias
    """
    print("\n" + "="*60)
    print("🩺 CONSULTA MÉDICA")
    print("="*60)
    print(f"\n❓ Pregunta: {question}\n")
    
    # Ejecutar RAG
    result = rag_chain.invoke({"query": question})
    
    # Mostrar documentos recuperados
    if verbose and 'source_documents' in result:
        print("📚 Conocimiento médico recuperado:")
        for i, doc in enumerate(result['source_documents'], 1):
            print(f"\n[{i}] Fuente: {doc.metadata.get('source', 'N/A')}")
            print(f"Contenido: {doc.page_content[:200]}...\n")
    
    print("="*60)
    print("💊 RESPUESTA")
    print("="*60)
    print(result['result'])
    print("\n" + "="*60)
    
    return result

print("✅ Función helper definida")

## 1️⃣2️⃣ EJEMPLO 1: Consulta Básica

In [ ]:
# Pregunta simple sobre diabetes
question1 = "What is the first-line treatment for type 2 diabetes?"

result1 = medical_query(question1)

## 1️⃣3️⃣ EJEMPLO 2: Caso Clínico Complejo

In [ ]:
# Caso clínico: Paciente diabético con disnea
clinical_case = """
A 65-year-old male patient with type 2 diabetes mellitus and hypertension 
presents with 3 days of dyspnea on exertion. 
Current medications: Losartan 50mg daily, Metformin 850mg daily.
What are the differential diagnoses and management considerations?
"""

result2 = medical_query(clinical_case)

## 1️⃣4️⃣ Integración con NER (Opcional)

Si tienes el NER ejecutado del notebook anterior:

In [ ]:
def enrich_query_with_ner(clinical_text, ner_results):
    """
    Enriquece una query con información del NER.
    
    Args:
        clinical_text: Texto clínico original
        ner_results: JSON output del notebook NER
    
    Returns:
        Query enriquecido con UMLS IDs y terminología normalizada
    """
    enriched_query = f"Clinical case: {clinical_text}\n\n"
    
    # Añadir enfermedades detectadas
    if 'ENFERMEDAD' in ner_results.get('entidades_por_categoria', {}):
        enriched_query += "Diagnosed conditions:\n"
        for disease in ner_results['entidades_por_categoria']['ENFERMEDAD']:
            nombre = disease.get('nombre_normalizado', disease.get('texto_original'))
            umls = disease.get('umls_id', '')
            enriched_query += f"- {nombre} (UMLS: {umls})\n"
    
    # Añadir medicamentos
    if 'MEDICAMENTO' in ner_results.get('entidades_por_categoria', {}):
        enriched_query += "\nCurrent medications:\n"
        for med in ner_results['entidades_por_categoria']['MEDICAMENTO']:
            nombre = med.get('nombre_normalizado', med.get('texto_original'))
            enriched_query += f"- {nombre}\n"
    
    # Añadir síntomas
    if 'SINTOMA' in ner_results.get('entidades_por_categoria', {}):
        enriched_query += "\nPresenting symptoms:\n"
        for symptom in ner_results['entidades_por_categoria']['SINTOMA']:
            nombre = symptom.get('nombre_normalizado', symptom.get('texto_original'))
            contexto = symptom.get('contexto', {})
            temp = contexto.get('temporalidad', '')
            enriched_query += f"- {nombre} ({temp})\n"
    
    return enriched_query

print("✅ Función de integración NER definida")

## 1️⃣5️⃣ Ejemplo con NER Integration

In [ ]:
# Simulación de output NER (reemplazar con tu NER real)
ner_example = {
    "entidades_por_categoria": {
        "ENFERMEDAD": [
            {
                "texto_original": "Diabetes Mellitus tipo 2",
                "umls_id": "C0011860",
                "nombre_normalizado": "Diabetes Mellitus, Type 2"
            },
            {
                "texto_original": "Hipertensión arterial",
                "umls_id": "C0020538",
                "nombre_normalizado": "Hypertension"
            }
        ],
        "MEDICAMENTO": [
            {
                "texto_original": "Losartán",
                "nombre_normalizado": "Losartan"
            },
            {
                "texto_original": "Metformina",
                "nombre_normalizado": "Metformin"
            }
        ],
        "SINTOMA": [
            {
                "texto_original": "disnea",
                "nombre_normalizado": "Dyspnea",
                "contexto": {"temporalidad": "actual"}
            }
        ]
    }
}

# Historia clínica original
clinical_text = "Paciente de 65 años con disnea de 3 días"

# Enriquecer query
enriched_query = enrich_query_with_ner(clinical_text, ner_example)

print("📝 Query enriquecido con NER:\n")
print(enriched_query)
print("\n" + "="*60)

# Consultar RAG con query enriquecido
result_ner = medical_query(f"{enriched_query}\n\nWhat is the most likely diagnosis and recommended management?")

## 1️⃣6️⃣ Guardar Vector Database (Para Reutilizar)

In [ ]:
# Guardar vector DB
db_path = "medical_vector_db"
vector_db.save_local(db_path)
print(f"✅ Vector database guardada en: {db_path}")

# Para cargar después:
# vector_db_loaded = FAISS.load_local(db_path, embeddings)

## 🎯 Próximos Pasos

### Mejoras Sugeridas:

1. **Más documentos médicos:**
   - Añadir más guías clínicas a `medical_knowledge/`
   - Protocolos locales de tu institución

2. **Filtrado por metadata:**
   - Especialidad médica
   - UMLS IDs específicos
   - Año de publicación

3. **Backend alternativo (Ollama):**
   - Instalar: `ollama pull meditron:7b`
   - Usar para privacidad total (local)

4. **Fine-tuning:**
   - Ajustar BioMistral con casos clínicos locales
   - Mejorar respuestas específicas a tu contexto

5. **UI con Streamlit:**
   - Interfaz web para uso interactivo
   - Integración completa NER + RAG

---

## 📚 Referencias

- **BioMistral:** https://huggingface.co/BioMistral/BioMistral-7B
- **PubMedBERT:** https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract
- **Langchain:** https://python.langchain.com/
- **NER Notebook:** `3_ner_avanzado_entitylinker.ipynb`